## Chapter 3

### 1. Coupon Feature Engineering.

Before we jump into feature engineering let me just mention that one should always do some data exploration. Within this dataset there is not much to explore that can drastically affect the techniques one would use afterwards. However, in the real world data exploration is mandatory. I will leave this to the reader. For example, it might be interesting to visualize the trends of sales per week/month, what are the features of the most popular coupons, age and gender distributions, etc. 

Moving into feature engineering, here I will manually generate a series of features. The reader might want to explore the use of this nice package [featuretools](https://github.com/Featuretools/featuretools), that can automate part of the process. 

To start with let's assume that we have information of the upcoming coupons with some time in advance and we can compute the features using the whole dataset.

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

from scipy import stats

inp_dir = "../datasets/Ponpare/data_processed"
out_dir = "../datasets/Ponpare/data_processed"

In [2]:
# Coupon features
df_coupons_train = pd.read_pickle(os.path.join(inp_dir, 'train', 'df_coupons_train.p'))
df_coupons_train['set_type'] = 2
df_coupons_valid = pd.read_pickle(os.path.join(inp_dir, 'valid', 'df_coupons_valid.p'))
df_coupons_valid['set_type'] = 1
df_coupons_test  = pd.read_pickle(os.path.join(inp_dir, 'test', 'df_coupons_test.p'))
df_coupons_test['set_type'] = 0
df_coupons = pd.concat(
    [df_coupons_train,df_coupons_valid,df_coupons_test],
    axis=0,
    ignore_index=True)

df_coupons.head()

,capsule_text,genre_name,price_rate,catalog_price,discount_price,dispfrom,dispend,dispperiod,validfrom,validend,validperiod,usable_date_mon,usable_date_tue,usable_date_wed,usable_date_thu,usable_date_fri,usable_date_sat,usable_date_sun,usable_date_holiday,usable_date_before_holiday,large_area_name,ken_name,small_area_name,coupon_id_hash,days_to_present,set_type
0,Food,Food,50,3000,1500,2011-07-08 12:00:00,2011-07-09 12:00:00,1,2011-07-10,2011-12-08,151.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,kanto,saitama,saitama,6b263844241eea98c5a97f1335ea82af,351,2
1,Food,Food,51,2080,1000,2011-07-01 12:00:00,2011-07-02 12:00:00,1,2011-07-03,2011-12-04,154.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,kanto,chiba,chiba,cc031f250e8bad1e24060263b9fc0ddd,358,2
2,Food,Food,50,7000,3500,2011-07-12 12:00:00,2011-07-15 12:00:00,3,2011-07-16,2012-01-11,179.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,kanto,chiba,chiba,ba5e9b7453ca52ff711635a5d2e8102d,347,2
3,Food,Food,50,3000,1500,2011-07-09 12:00:00,2011-07-11 12:00:00,2,2011-07-12,2011-12-01,142.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,kanto,chiba,chiba,3e1ffbedca3569f9e8032d401e8cb4e6,350,2
4,Food,Food,50,2000,1000,2011-07-05 12:00:00,2011-07-06 12:00:00,1,2011-07-07,2011-12-30,176.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,kanto,chiba,chiba,782934b6c815b4030ea204eef7d4a734,354,2


In [3]:
# columns with NaN
has_nan = df_coupons.isnull().any(axis=0)
has_nan = [df_coupons.columns[i] for i in np.where(has_nan)[0]]

print(has_nan)

['validfrom', 'validend', 'validperiod', 'usable_date_mon', 'usable_date_tue', 'usable_date_wed', 'usable_date_thu', 'usable_date_fri', 'usable_date_sat', 'usable_date_sun', 'usable_date_holiday', 'usable_date_before_holiday']


#### NaN as another category

Interestingly, all features with NaN are time-related. There are a number of ways one could replace the NaNs (e.g. using the mode in the column). Here I will treat NaNs as if they were another category for all but the first 3 columns (Read Below).

In [4]:
for col in has_nan[3:]:
    df_coupons[col] = df_coupons[col].fillna(3).astype('int')

Let's have a quick look to the the feature validperiod

In [5]:
df_coupons[df_coupons.validperiod.isna()]['capsule_text'].value_counts()

Delivery service         5821
Lesson                    260
Gift card                  52
Correspondence course      12
Other                       1
Leisure                     1
Name: capsule_text, dtype: int64

On the other hand, 6147 of the 19413 `validperiod` values are NaN. 5821 of those are of type "Delivery service". In the case of the "Correspondence course" *all* observations have `validperiod` value NaN and "Other" and "Leisure" only have one entry with NaN. There is a chance that those coupons with no `validperiod` "last forever". 

We will explore here two possible methods to fill NaN for the `validperiod` column

#### validperiod Method 1: NaN as another category

Note: maybe one could to consider coupons with a `validperiod` of 0 as a class. For now, we will simply use quatiles

In [6]:
df_coupons['validperiod_method1_cat'], validperiod_bins_method1 = pd.qcut(df_coupons['validperiod'], q=4, labels=[0,1,2,3], retbins=True)
df_coupons.validperiod_method1_cat.cat.add_categories([4], inplace=True)
df_coupons['validperiod_method1_cat'].fillna(4, inplace=True)

#### validperiod Method 2: replace NaN first and then into categorical

In [7]:
validperiod_mode = df_coupons.validperiod.mode().values[0]
caps_vals = list(df_coupons[df_coupons.validperiod.isna()]['capsule_text'].value_counts().index)
for ct in caps_vals:
    non_nan_values = df_coupons[(df_coupons.capsule_text == ct) &
        (~df_coupons.validperiod.isna())]['validperiod'].values
    nan_idx = list(df_coupons[(df_coupons.capsule_text == ct) &
        (df_coupons.validperiod.isna())].index)
    # Note that '50' is rather arbitray 
    if (len(nan_idx)>50) & (non_nan_values.size>0):
        replace_vals = np.random.choice(non_nan_values, len(nan_idx))
        df_coupons.loc[nan_idx, 'validperiod'] = replace_vals
    elif (len(nan_idx)<=50) & (non_nan_values.size>0):
        median = np.median(non_nan_values)
        df_coupons.loc[nan_idx, 'validperiod'] = median
    elif non_nan_values.size==0:
        df_coupons.loc[nan_idx, 'validperiod'] = validperiod_mode
df_coupons['validperiod_method2_cat'] = pd.cut(df_coupons['validperiod'],
    bins=validperiod_bins_method1, labels=[0,1,2,3], include_lowest=True)
df_coupons['validperiod'] = df_coupons['validperiod'].astype('int')

In [8]:
# let's save the mappings. This dictionary will be fundamental in moving forward
dict_of_mappings = {}
dict_of_mappings['validperiod_cat'] = validperiod_bins_method1

#### validfrom and validend

In [9]:
# Let's now take care of validfrom and validend
valid_cols = ['validfrom', 'validend']
for col in valid_cols:

    valid_mode = df_coupons[col].dt.dayofweek.mode().values[0]

    # Method 1: NaN as another category
    new_colname_1 = col+"_method1_cat"
    df_coupons[new_colname_1] = df_coupons[col].dt.dayofweek
    df_coupons[new_colname_1] = df_coupons[new_colname_1].fillna(7).astype('int')

    # Method 2: replace and to category
    new_colname_2 = col+"_method2_cat"
    df_coupons[new_colname_2] = df_coupons[col].dt.dayofweek
    caps_vals = list(df_coupons[df_coupons[new_colname_2].isna()]['capsule_text'].value_counts().index)
    for ct in caps_vals:
        non_nan_values = df_coupons[(df_coupons.capsule_text == ct) &
            (~df_coupons[new_colname_2].isna())][new_colname_2].values
        nan_idx = list(df_coupons[(df_coupons.capsule_text == ct) &
            (df_coupons[new_colname_2].isna())].index)
        if (len(nan_idx)>50) & (non_nan_values.size>0):
            replace_vals = np.random.choice(non_nan_values, len(nan_idx))
            df_coupons.loc[nan_idx, new_colname_2] = replace_vals
        elif (len(nan_idx)<=50) & (non_nan_values.size>0):
            mode = stats.mode(non_nan_values).mode[0]
            df_coupons.loc[nan_idx, new_colname_2] = mode
        elif non_nan_values.size==0:
            df_coupons.loc[nan_idx, new_colname_2] = valid_mode
    df_coupons[new_colname_2] = df_coupons[new_colname_2].astype('int')

#### dispfrom, dispend and dispperiod

In [10]:
df_coupons['dispfrom_cat'] = df_coupons.dispfrom.dt.dayofweek
df_coupons['dispend_cat'] = df_coupons.dispend.dt.dayofweek
# also add dispperiod as categorical
df_coupons['dispperiod_cat'], dispperiod_bins = pd.qcut(df_coupons.dispperiod, q=4, labels=[0,1,2,3], retbins=True)
dict_of_mappings['dispperiod_cat'] = dispperiod_bins

#### price related features

In [11]:
df_coupons['price_rate_cat'], price_rate_bins = pd.qcut(df_coupons['price_rate'], q=3, labels=[0,1,2], retbins=True)
df_coupons['catalog_price_cat'], catalog_price_bins = pd.qcut(df_coupons['catalog_price'], q=3, labels=[0,1,2], retbins=True)
df_coupons['discount_price_cat'], discount_price_bins = pd.qcut(df_coupons['discount_price'], q=3, labels=[0,1,2], retbins=True)

In [12]:
# Update dictionary of mappings
dict_of_mappings['price_rate_cat'] = price_rate_bins
dict_of_mappings['catalog_price_cat'] = catalog_price_bins
dict_of_mappings['discount_price_cat'] = discount_price_bins

#### Final encoding

In [13]:
# Finally let's LabelEncode some additional features. I will do it
# manually since I want to keep the mappings (as opposed as using LabelEncoder in sklearn)
le_cols = ['capsule_text', 'genre_name', 'large_area_name', 'ken_name', 'small_area_name']
for col in le_cols:
    dict_of_mappings[col+"_cat"] = {}
    values = list(df_coupons[col].unique())
    labels = list(np.arange(len(values)))
    dict_of_mappings[col+"_cat"] = dict(zip(values,labels))
    df_coupons[col+'_cat'] = df_coupons[col].replace(dict_of_mappings[col+"_cat"])

drop_cols = le_cols + ['dispfrom', 'dispend', 'validfrom', 'validend', 'days_to_present']
df_coupons.drop(drop_cols, axis=1, inplace=True)

# for convenience later, let's add the suffix "cat" to usable_date columns
usable_date_cols_dict = {c:c+"_cat" for c in df_coupons.columns if 'usable' in c}
df_coupons.rename(index=str, columns=usable_date_cols_dict, inplace=True)

# split back to the original datasets
df_coupons_train = (df_coupons[df_coupons.set_type == 2]
    .drop('set_type', axis=1)
    .reset_index(drop=True))
df_coupons_valid = (df_coupons[df_coupons.set_type == 1]
    .drop('set_type', axis=1)
    .reset_index(drop=True))
df_coupons_test = (df_coupons[df_coupons.set_type == 0]
    .drop('set_type', axis=1)
    .reset_index(drop=True))

In [14]:
# Let's have a look to the resulting df
df_coupons.head()

,price_rate,catalog_price,discount_price,dispperiod,validperiod,usable_date_mon_cat,usable_date_tue_cat,usable_date_wed_cat,usable_date_thu_cat,usable_date_fri_cat,usable_date_sat_cat,usable_date_sun_cat,usable_date_holiday_cat,usable_date_before_holiday_cat,coupon_id_hash,set_type,validperiod_method1_cat,validperiod_method2_cat,validfrom_method1_cat,validfrom_method2_cat,validend_method1_cat,validend_method2_cat,dispfrom_cat,dispend_cat,dispperiod_cat,price_rate_cat,catalog_price_cat,discount_price_cat,capsule_text_cat,genre_name_cat,large_area_name_cat,ken_name_cat,small_area_name_cat
0,50,3000,1500,1,151,1,1,1,1,0,0,1,1,0,6b263844241eea98c5a97f1335ea82af,2,2,2,6,6,3,3,4,5,0,0,0,0,0,0,0,0,0
1,51,2080,1000,1,154,1,1,1,1,1,1,1,1,1,cc031f250e8bad1e24060263b9fc0ddd,2,2,2,6,6,6,6,4,5,0,1,0,0,0,0,0,1,1
2,50,7000,3500,3,179,0,1,1,1,1,1,1,1,1,ba5e9b7453ca52ff711635a5d2e8102d,2,3,3,5,5,2,2,1,4,1,0,1,1,0,0,0,1,1
3,50,3000,1500,2,142,1,1,1,1,0,0,1,1,1,3e1ffbedca3569f9e8032d401e8cb4e6,2,2,2,1,1,3,3,5,0,0,0,0,0,0,0,0,1,1
4,50,2000,1000,1,176,1,1,0,1,0,0,1,1,0,782934b6c815b4030ea204eef7d4a734,2,2,2,3,3,4,4,1,2,0,0,0,0,0,0,0,1,1


#### save

In [15]:
# save files
df_coupons_train.to_pickle(os.path.join(out_dir,"train","df_coupons_train_feat.p"))
df_coupons_valid.to_pickle(os.path.join(out_dir,"valid","df_coupons_valid_feat.p"))
df_coupons_test.to_pickle(os.path.join(out_dir,"test","df_coupons_test_feat.p"))

# save dictionary
pickle.dump(dict_of_mappings, open(os.path.join(out_dir, 'dict_of_mappings.p'), 'wb') )